In [ ]:
from ltr.clickmodels.session import build

# Sessions for search queries "rambo" and "star trek"
sessions = build([
      ('rambo',     ((1, True), (2, False), (3, True), (0, False))),
      ('star trek', ((5, False), (2, True), (3, True), (0, False))),
      ('rambo',     ((1, False), (2, False), (3, True), (0, False))),
      ('star trek', ((1, False), (2, False), (3, False), (9, True))),
      ('rambo',     ((9, False), (2, False), (1, True), (0, True))),
      ('star trek', ((6, True), (2, False), (3, True), (1, False))),
      ('rambo',     ((7, False), (4, True), (1, False), (3, False))),
      ('star trek', ((8, True), (2, False), (3, True), (1, False))),
      ('rambo',     ((1, False), (4, True), (2, False), (3, False))),
      ('star trek', ((7, True), (4, False), (5, True), (1, True)))
    ])

In [ ]:
def by_query(judgment_dict):
    from collections import defaultdict
    by_query_ranked = defaultdict(list)
    
    for (query, doc), score in judgment_dict.items():
        by_query_ranked[query].append((doc,score))
        
    for query, items in by_query_ranked.items():
        items.sort(key=lambda score_doc: score_doc[1], reverse=True)
        
    return by_query_ranked

def just_doc_ids(docs):
    return [doc[0] for doc in docs]

# COEC 
(Clicks Over Expected Clicks)

Clicks relative to CTR at that position. > 1 means above average, < 1 means below average

In [ ]:
from ltr.clickmodels.coec import coec

ctrs_by_rank = [0.15,0.15,0.15,0.03]

model = coec(ctrs_by_rank, sessions)
coec_judgments = by_query(model.coecs)

# PBM

(Position-Based model)

We learn probability of a rank being examined and the attractiveness of a query-doc pair

In [ ]:
from ltr.clickmodels.pbm import position_based_model
model = position_based_model(sessions, rounds=50)
pbm_judgments = by_query(model.attracts)

In [ ]:
model.ranks

In [ ]:
pbm_judgments['rambo']

### Compare

In [ ]:
from ltr.helpers.tau import tau
from ltr.helpers.tau import avg_tau

rambo1 = just_doc_ids(coec_judgments['rambo'])
rambo1


In [ ]:
rambo2 = just_doc_ids(pbm_judgments['rambo'])
rambo2


In [ ]:
avg_tau(rambo1, rambo2, at=7)

In [ ]:
star_trek1 = just_doc_ids(coec_judgments['star trek'])
star_trek1

In [ ]:
star_trek2 = just_doc_ids(pbm_judgments['star trek'])
star_trek2

In [ ]:
avg_tau(star_trek1, star_trek2, at=10)

# UBM

(User Browse Model)

Variant of PBM model that computes examination as a function of this position and the last clicked position.

In [ ]:
from ltr.clickmodels.ubm import user_browse_model

# Sessions for search queries "rambo" and "star trek"
sessions2 = build([
      ('rambo',     ((1, True), (2, False))),
      ('star trek', ((5, False), (2, True))),
      ('rambo',     ((1, True), (2, False))),
      ('star trek', ((5, False), (2, True)))
    
    ])

model = user_browse_model(sessions2, rounds=50)
model.attracts

In [ ]:
model.ranks

# Cascade Model

A variant of the model that computes relevance based on clicks over times the article appeared at or before the first click

In [ ]:
from ltr.clickmodels.cascade import cascade_model
model = cascade_model(sessions)

In [ ]:
model.attracts

# SDBN

Simplified version of the Dynamic Bayesian Network, that computes satisfaction and attractiveness. Satisfaction is the times this item was clicked relative to all sessions that item was clicked. Attractiveness how often this item was clicked relative to all sessions with that query/doc

In [ ]:
from ltr.clickmodels.sdbn import sdbn
model = sdbn(sessions)

In [ ]:
model.attracts

In [ ]:
model.satisfacts